# Dependencies Installation

In [29]:
!pip install geopandas simplekml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Optional Download File

In [30]:
import requests

def download_shapefile(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print("Download Finished.")
    else:
        print("Failed to Download File")


In [31]:
download_file = False #@param {type:"boolean"}
url = '' #@param {type:"string"}
save_path = '' #@param {type:"string"}

if download_file:
  download_shapefile(url, save_path)

# Convert Shapefile to Kml

In [32]:
import geopandas as gpd
import simplekml

In [69]:
def shapefile_to_kml(shapefile_path=None, output_kml_path=None):
    if shapefile_path==None or '': raise ValueError("None Type as Shapefile path.")
    gdf = gpd.read_file(shapefile_path)

    kml = simplekml.Kml()

    for index, row in gdf.iterrows():
        geom = row['geometry']
        kml_geom = geom.__geo_interface__

        geometry_type = kml_geom['type']
        field_name = gdf.columns[0]
        placemark = kml.newmultigeometry(name=row[field_name])
        if geometry_type == 'Point':
            coordinates = kml_geom['coordinates']
            placemark.newpoint(coords=[coordinates])
        elif geometry_type == 'LineString':
            coordinates = kml_geom['coordinates']
            linestring = placemark.newlinestring()
            linestring.coords = coordinates
        elif geometry_type == 'Polygon':
            coordinates = kml_geom['coordinates'][0]
            polygon = placemark.newpolygon()
            polygon.outerboundaryis = coordinates

        for field in gdf.columns:
            if field != 'geometry':
                value = str(row[field])
                placemark.extendeddata.schemadata.newsimpledata(field, value)
    kml.save(output_kml_path)

In [71]:
shapefile_path = "/content/lm_aisp_2019.shp" #@param {type:"string"}
output_kml_path = '/content/aisp.kml'  #@param {type:"string"}
shapefile_to_kml(shapefile_path=shapefile_path, output_kml_path=output_kml_path)